# K-fold Cross Validation

## Import

In [1]:
import torch

from torchvision.datasets import MNIST
from torchvision import transforms
from torch.utils.data import DataLoader, ConcatDataset, SubsetRandomSampler
import torch.nn as nn
import torch.optim as opt
from torch.autograd import Variable
from sklearn.model_selection import KFold

print(torch.__version__)

1.9.0+cu102


## Set the Device

In [2]:
torch.cuda.get_device_name()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Dataset - MNIS

In [3]:
train_dataset = MNIST(root = './data', train=True, download=True, transform=transforms.ToTensor())
test_dataset= MNIST(root = './data', train=False, download=False, transform=transforms.ToTensor())

dataset = ConcatDataset([train_dataset, test_dataset])

/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


## Model - CNN

In [4]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(stride=2, kernel_size=2)
        )
        
        self.dense = nn.Sequential(
            nn.Linear(in_features=14*14*128, out_features=1024),
            nn.ReLU(),
            nn.Linear(1024, 10)
        )


    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
                m.reset_parameters()

    def forward(self, x):
        output = self.conv_layers(x)
        output = output.view(-1, 14*14*128)
        output = self.dense(output)
        return output

In [5]:
# model = CNN()
# model.init_weights()
# model.to(device)

## Run K-Fold Cross Validation

In [6]:
k_folds = 5
num_epochs = 3
learning_rate = 0.001

# loss function
loss_func = nn.CrossEntropyLoss()

# for fold results
results = {}

# set fixed random seed
torch.manual_seed(87)

# define the k-fold cross validator
kfold = KFold(n_splits=k_folds, shuffle=True)

In [8]:
# k-fold cross validation model evaluation

for fold, (train_ids, test_ids) in enumerate(kfold.split(test_dataset)):
    
    print(f'\n============== Fold: {fold+1} Start ==============')
    # Sample elements randomly from a given list of ids.
    train_subsampler = SubsetRandomSampler(train_ids)
    test_subsampler = SubsetRandomSampler(test_ids)

    # define data loader for training and testing data in this fold
    trainloader = DataLoader(
        dataset,
        batch_size=64,
        sampler=train_subsampler
    )

    testloader = DataLoader(
        dataset,
        batch_size=64,
        sampler=test_subsampler
    )

    # init the model
    model = CNN()
    model.init_weights()
    model.to(device)

    # Initialize optimizer
    optimizer = opt.Adam(model.parameters(), lr=learning_rate)

    # run training loop
    for epoch in range(0, num_epochs):
        this_epoch_loss = 0.0
        for idx, (images, labels) in enumerate(trainloader):
            
            images = Variable(images).to(device)
            labels = Variable(labels).to(device)

            # zero the gradients
            optimizer.zero_grad()

            # perform forward pass
            outputs = model(images)

            # compute loss
            loss = loss_func(outputs, labels)

            # backward pass
            loss.backward()
            # optimization
            optimizer.step()
            
            this_epoch_loss += loss.item()
        print(f'- Epoch {epoch+1} loss: %.3f' % (this_epoch_loss/len(trainloader.sampler)) )

    # Process is complete.
    print('\n- Training process has finished. \n- Saving trained model.')
    # Saving the model
    save_path = f'./model-fold-{fold}.pth'
    torch.save(model.state_dict(), save_path)

    # evaluation for this fold
    correct, total = 0, 0
    with torch.no_grad():
        for idx, (images, labels) in enumerate(testloader):
            images = Variable(images.to(device))
            outputs = model(images)

            _, pred = torch.max(outputs.data, 1)
            correct += (pred == labels.to(device)).sum()
            total += labels.size(0)

        print('\n* Accuracy for Fold %d: %.3f%%' %(fold+1, 100.0 * float(correct)/float(total)))
        results[fold+1] = 100.0 * float(correct)/float(total)

    print(f'=============== Fold: {fold+1} End ===============')

# Print fold results
print(f'\nK-fold Cross Validtaion results for {k_folds} folds')
sum = 0.0
for key, value in results.items():
    print(f'* Fold {key}: {value} %')
    sum += value
    
print(f'* Average: {sum/len(results.items())} %')


============== Fold: 1 Start ==============
- Epoch 1 loss: 0.006
- Epoch 2 loss: 0.001
- Epoch 3 loss: 0.001

- Training process has finished. 
- Saving trained model.

* Accuracy for Fold 1: 97.150%
=============== Fold: 1 End ===============

============== Fold: 2 Start ==============
- Epoch 1 loss: 0.007
- Epoch 2 loss: 0.001
- Epoch 3 loss: 0.001

- Training process has finished. 
- Saving trained model.

* Accuracy for Fold 2: 97.600%
=============== Fold: 2 End ===============

============== Fold: 3 Start ==============
- Epoch 1 loss: 0.008
- Epoch 2 loss: 0.002
- Epoch 3 loss: 0.001

- Training process has finished. 
- Saving trained model.

* Accuracy for Fold 3: 97.250%
=============== Fold: 3 End ===============

============== Fold: 4 Start ==============
- Epoch 1 loss: 0.007
- Epoch 2 loss: 0.002
- Epoch 3 loss: 0.001

- Training process has finished. 
- Saving trained model.

* Accuracy for Fold 4: 97.650%
=============== Fold: 4 End ===============

============== 